In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
os.chdir("..")

In [4]:
import torch
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from tqdm.auto import tqdm
from datasets import load_dataset
from cluster_intrep_repo.utils import initialize_tokenizer, tokenize_blocksworld_generation, THINK_TOKEN



os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

compute_dtype = torch.bfloat16
device   = 'cuda'
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

In [5]:
tokenizer = initialize_tokenizer(model_id)

In [6]:
blocksworld_type = "4-blocks"

dataset = load_dataset(f"dmitriihook/deepseek-r1-qwen-32b-planning-{blocksworld_type}")["train"]

In [7]:
labels_dataset = load_dataset(f"dmitriihook/blocksworld-4-final-labels")["train"]

blocksworld-4-final-labels.json:   0%|          | 0.00/416k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [8]:
model     = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=compute_dtype, attn_implementation="sdpa", device_map="auto")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
n_rows = 1500

In [10]:
from collections import defaultdict

# [src; dest]

layer_hidden_states = defaultdict(list)

n_last_layers = 10

for row in tqdm(dataset.select(range(n_rows))):
    generation = row["generation"]

    if "</think>" not in generation:
        for j in range(n_last_layers):
            layer_hidden_states[j].append(None) 
        continue

    chat = tokenize_blocksworld_generation(tokenizer, row)

    # think_pos = torch.where(chat.squeeze() == THINK_TOKEN)[0]

    with torch.no_grad():
        outputs = model(chat.to(device), output_hidden_states=True)

        for j in range(n_last_layers):
            hidden_states = outputs.hidden_states[-1 - j]
            layer_hidden_states[j].append(hidden_states[0].to(torch.float16).cpu().numpy())

  0%|          | 0/1500 [00:00<?, ?it/s]

In [11]:
for j in range(n_last_layers):
    # layer_hidden_states[j] = [x for x in layer_hidden_states[j] if x is not None]
    print(len(layer_hidden_states[j]))

1500
1500
1500
1500
1500
1500
1500
1500
1500
1500


In [12]:
import re
from collections import defaultdict

def parse_blocks(text):
    initial_state = []
    goal_state = []
    
    # Extract the initial conditions and goal state
    initial_match = re.search(r'As initial conditions I have that:(.*?)My goal is for the following to be true:', text, re.DOTALL)
    goal_match = re.search(r'My goal is for the following to be true:(.*?)\n\n', text, re.DOTALL)

    if initial_match:
        initial_conditions = re.findall(r'Block [A-Z] is on top of Block [A-Z]', initial_match.group(1))
        init_table_blocks = re.findall(r'Block ([A-Z]) is on the table', initial_match.group(1))
        initial_state = process_conditions(initial_conditions)

    
    if goal_match:
        goal_conditions = re.findall(r'Block [A-Z] is on top of Block [A-Z]', goal_match.group(1))
        goal_table_blocks = re.findall(r'Block ([A-Z]) is on the table', goal_match.group(1))
        goal_state = process_conditions(goal_conditions)

    
    return (initial_state, init_table_blocks), (goal_state, goal_table_blocks)

def process_conditions(conditions):
    pairs = {}
    
    for cond in conditions:
        block, below = re.findall(r'Block ([A-Z])', cond)
        pairs[block] = below
    
    return pairs


item = dataset[2]["query"]
stmt = item.split("[STATEMENT]")[-1].strip()

initial_state, goal_state = parse_blocks(stmt)
initial_state, goal_state

(({'B': 'C', 'C': 'D'}, ['A', 'D']), ({'A': 'C', 'C': 'D', 'D': 'B'}, []))

In [13]:
def state_to_pairs(state, all_blocks):
    pairs, _ = state
    below = {}

    for block, below_block in pairs.items():
        below[block] = below_block

    for block in all_blocks:
        if block not in below:
            below[block] = "table"

    above = {}

    for block, below_block in below.items():
        if below_block != "table":
            above[below_block] = block

    for block in all_blocks:
        if block not in above:
            above[block] = "sky"
    
    return above, below

In [14]:
def collect_all_blocks(initial_state):
    all_blocks = list(initial_state[0].keys())
    all_blocks.extend(initial_state[1])
    all_blocks.extend(initial_state[0].values())
    return list(set(all_blocks))

In [15]:
all_blocks = collect_all_blocks(initial_state)

state_to_pairs(initial_state, all_blocks)

({'C': 'B', 'D': 'C', 'A': 'sky', 'B': 'sky'},
 {'B': 'C', 'C': 'D', 'A': 'table', 'D': 'table'})

In [16]:
from typing import Optional

def apply_action(action: list[str], state: tuple[dict, dict, Optional[str]]) -> Optional[tuple[dict, dict, Optional[str]]]: 
    above, below, hand = state
    # print(action)

    above = above.copy()
    below = below.copy()

    action_type, blocks = action[0], action[1:]

    if action_type == "pick up":
        if hand is not None:
            return None
        block = blocks[0]
        above_block = above[block]

        if above_block != "sky":
            return None
        
        below_block = below[block]
        if below_block != "table":
            above[below_block] = "sky"
            below[block] = "table"
        
        hand = block

    elif action_type == "put down":
        if hand is None:
            return None
        
        if hand != blocks[0]:
            return None
        
        block = blocks[0]
        hand = None
    elif action_type == "unstack":
        if hand is not None:
            return None
        
        block1, block2 = blocks
        if above[block1] != "sky":
            return None
        if below[block1] != block2:
            return None
        
        above[block2] = "sky"
        below[block1] = "table"

        hand = block1
    elif action_type == "stack":
        block1, block2 = blocks

        if hand != block1:
            return None

        if above[block2] != "sky":
            return None
        
        above[block2] = block1
        below[block1] = block2
        hand = None

    return above, below, hand

In [121]:
def find_index_before_block(text):
    text = text.lower()
    regex = r"\d+\.\s*(?:unstack|put down|pick up|stack)\s+(?:block\s+)?([a-z])"

    match = re.search(regex, text)
    
    return match.start(1) if match else None

training_data = []
for label_item in tqdm(labels_dataset.select(range(n_rows))):
    i = label_item["index"]
    if label_item["label"] is None:
        continue

    row = dataset[i]
    actions = label_item["label"]["final_plan"]
    # print(actions)
    generation: str = row["generation"]

    if "</think>" not in generation:
        continue

    head, tail = generation.split("</think>")
    phrase = label_item["label"]["starting_phrase"]

    phrase_end = head.rfind(phrase)

    text = head[:phrase_end + len(phrase)]
    _text = head[phrase_end + len(phrase):]
    block_idx = find_index_before_block(_text)

    if block_idx is None:
        continue

    text += _text[:block_idx]

    group = []

    stmt = row["query"].split("[STATEMENT]")[-1].strip()
    initial_state, goal_state = parse_blocks(stmt)

    all_blocks = collect_all_blocks(initial_state)
    initial_state = state_to_pairs(initial_state, all_blocks)
    goal_state = state_to_pairs(goal_state, all_blocks)

    current_state = (initial_state[0], initial_state[1], None)

    group = []

    for action in actions:
        if current_state is None:
            continue

        try:
            next_state = apply_action(action, current_state)
        except Exception as e:
            print(e)
            next_state = None
        if next_state is not None:
            group.append({
                "idx": i,
                "action": action,
                "before_state": current_state,
                "after_state": next_state
            })

        current_state = next_state
    
    tokens = tokenize_blocksworld_generation(tokenizer, row, text)[0]


    training_data.append({
        "idx": i,
        "initial_state": initial_state,
        "goal_state": goal_state,
        "actions": actions,
        "group": group,
        "pos": len(tokens),
        "phrase": phrase
    })


  0%|          | 0/1500 [00:00<?, ?it/s]

'the table'
'table'
'table'
not enough values to unpack (expected 2, got 1)
'table'


In [123]:
item = training_data[11]

row = dataset[item["idx"]]

tokens = tokenize_blocksworld_generation(tokenizer, row)[0]

tokenizer.decode(tokens[item["pos"]:item["pos"]+5])


' C. (D is'

In [124]:
training_data[0]

{'idx': 0,
 'initial_state': ({'A': 'B', 'D': 'C', 'B': 'sky', 'C': 'sky'},
  {'B': 'A', 'C': 'D', 'A': 'table', 'D': 'table'}),
 'goal_state': ({'D': 'A', 'C': 'B', 'B': 'D', 'A': 'sky'},
  {'A': 'D', 'B': 'C', 'D': 'B', 'C': 'table'}),
 'actions': [['unstack', 'C', 'D'],
  ['put down', 'C'],
  ['unstack', 'B', 'A'],
  ['stack', 'B', 'C'],
  ['pick up', 'D'],
  ['stack', 'D', 'B'],
  ['pick up', 'A'],
  ['stack', 'A', 'D']],
 'group': [{'idx': 0,
   'action': ['unstack', 'C', 'D'],
   'before_state': ({'A': 'B', 'D': 'C', 'B': 'sky', 'C': 'sky'},
    {'B': 'A', 'C': 'D', 'A': 'table', 'D': 'table'},
    None),
   'after_state': ({'A': 'B', 'D': 'sky', 'B': 'sky', 'C': 'sky'},
    {'B': 'A', 'C': 'table', 'A': 'table', 'D': 'table'},
    'C')},
  {'idx': 0,
   'action': ['put down', 'C'],
   'before_state': ({'A': 'B', 'D': 'sky', 'B': 'sky', 'C': 'sky'},
    {'B': 'A', 'C': 'table', 'A': 'table', 'D': 'table'},
    'C'),
   'after_state': ({'A': 'B', 'D': 'sky', 'B': 'sky', 'C': 'sky'

In [125]:
n_blocks = int(dataset[n_rows - 1]["instance_id"].split("_")[0])
n_blocks

4

In [126]:
from torch.utils.data import Dataset

act2int = {
    "put down": 0,
    "pick up": 1,
    "stack": 2,
    "unstack": 3
}

def block2int(block):
    if block == "table":
        return n_blocks
    if block == "sky":
        return n_blocks + 1
    
    return ord(block) - ord("A")

def int2block(i):
    if i == n_blocks:
        return "table"
    if i == n_blocks + 1:
        return "sky"
    
    return chr(i + ord("A"))

n_prev_tokens = 10

def state_to_label(state):
    above, below, hand = state
    label = np.zeros((n_blocks * 2, ), dtype=np.int64)

    for block, below_block in below.items():
        label[block2int(block)] = block2int(below_block)
    for block, above_block in above.items():
        label[block2int(block) + n_blocks] = block2int(above_block)

    return label



class StepProbeDataset(Dataset):
    def __init__(self, items, hidden_states, n_layer, jump):
        self.items = items
        self.hidden_states = hidden_states
        self.n_layer = n_layer
        self.jump = jump

    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        item = self.items[idx]
        hidden_states = self.hidden_states[self.n_layer][item["idx"]]
        pos = item["pos"]

        above, below, hand = item["group"][self.jump]["after_state"]

        return {
            "input": hidden_states[pos:pos+20],
            "labels": state_to_label((above, below, hand))
        }


In [127]:
import random

def make_training_data(jump=0, train_test_split=0.9):
    # expanded_training_data = []

    # for group in training_data:
    #     group = group["group"]
    #     for action1, action2 in zip(group, group[jump:]):
    #         if len(action1["action"][1]) < 1:
    #             continue
    #         expanded_training_data.append((action1, action2))
    #         # continue

    filtered_data = [x for x in training_data if len(x["group"]) > jump]
    filtered_data = [x for x in filtered_data if layer_hidden_states[0][x["idx"]] is not None]

    random.shuffle(filtered_data)

    n_train = int(len(filtered_data) * train_test_split)

    train_items = filtered_data[:n_train]
    test_items = filtered_data[n_train:]

    train_dataset = StepProbeDataset(train_items, layer_hidden_states, 0, jump=jump)
    test_dataset = StepProbeDataset(test_items, layer_hidden_states, 0, jump=jump)

    return train_dataset, test_dataset

In [128]:
class StepProbe(torch.nn.Module):
    def __init__(self, input_size, hidden_size, n_blocks):
        super().__init__()
        # self.fc = torch.nn.Linear(input_size, hidden_size)
        # self.fc2 = torch.nn.Linear(hidden_size, n_blocks * (n_blocks + 2) * 2)
        self.fc2 = torch.nn.Linear(input_size, n_blocks * (n_blocks + 2) * 2)
        # self.dropout = torch.nn.Dropout(0.1)
        
    def forward(self, x):
        # x = self.fc(x)
        # x = torch.nn.functional.relu(x)
        # x = self.dropout(x)
        x = self.fc2(x[:, 0])
        return x.view(-1, n_blocks + 2, n_blocks * 2)

In [129]:
class GRUProbe(torch.nn.Module):
    def __init__(self, input_size, hidden_size, n_blocks):
        super().__init__()
        self.gru = torch.nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, n_blocks * (n_blocks + 2) * 2)
        
    def forward(self, x):
        x /= 2
        x, _ = self.gru(x)
        x = x[:, -1]
        x = self.fc(x)
        return x.view(-1, n_blocks + 2, n_blocks * 2)

In [130]:
jumps = list(range(6))

jump_datasets = {
    jump: make_training_data(jump) for jump in jumps
}

for jump, (train, test) in jump_datasets.items():
    print(jump, len(train), len(test))

0 1221 136
1 1189 133
2 1168 130
3 1147 128
4 1098 122
5 1079 120


In [131]:
n_dim = 5120

probes = {jump: StepProbe(n_dim, 500, n_blocks).to(device) for jump in jumps}

In [132]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from sklearn.metrics import f1_score

def train_probe(probe, train_dataset, test_dataset, patience=100):
    optimizer = AdamW(probe.parameters(), lr=1e-3, weight_decay=1e-2)
    criterion = CrossEntropyLoss(weight=torch.tensor([1.0] * n_blocks + [0.1] * 2).to(device))
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    n_epochs = 500
    best_f1 = float('inf')
    early_stop_counter = 0
    
    for epoch in range(n_epochs):
        probe.train()
        total_loss = 0
        n_samples = 0

        for batch in train_loader:
            optimizer.zero_grad()
            input = batch["input"].to(device).float()
            labels = batch["labels"].to(device)
            
            output = probe(input)

            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * len(batch["input"])
            n_samples += len(batch["input"])

        avg_train_loss = total_loss / n_samples
        
        # Evaluation
        probe.eval()
        with torch.no_grad():
            block_wise_hits = np.zeros((n_blocks * 2), dtype=np.int64)
            total = 0  
            val_loss = 0
            all_preds = []
            all_labels = []
            
            for batch in test_loader:
                input = batch["input"].to(device).float()
                labels = batch["labels"].to(device)
                
                output = probe(input)

                loss = criterion(output, labels)
                val_loss += loss.item() * len(batch["input"])

                preds = output.argmax(dim=-2)  # Assuming classification task
                hits = (preds == labels)
                
                block_wise_hits += hits.sum(dim=0).cpu().numpy()
                total += len(labels)
                
                all_preds.append(preds.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
            
            block_wise_hits = block_wise_hits / total
            
            all_preds = np.concatenate(all_preds)
            all_labels = np.concatenate(all_labels)
            
            # Compute F1 score block-wise
            block_wise_f1 = np.zeros(n_blocks * 2)
            for i in range(n_blocks * 2):
                block_wise_f1[i] = f1_score(all_labels[:, i], all_preds[:, i], average='macro')
            
            avg_f1 = block_wise_f1.mean()

            val_loss /= total
            
            print(f"Epoch {epoch}, Train Loss: {avg_train_loss:.4f}, Hits: {block_wise_hits.mean():.4f}, F1: {avg_f1:.4f}, Val Loss: {val_loss:.4f}")
        
            # Early Stopping Check
            if avg_f1 > best_f1:
                best_f1 = avg_f1
                early_stop_counter = 0
            else:
                early_stop_counter += 1
            
            if early_stop_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break
    
    return block_wise_hits, block_wise_f1


In [133]:
for jump, (train, test) in jump_datasets.items():
    print(jump)
    print(train_probe(probes[jump], train, test))


0
Epoch 0, Train Loss: 2.5821, Hits: 0.2105, F1: 0.1392, Val Loss: 1.9838
Epoch 1, Train Loss: 1.5632, Hits: 0.3006, F1: 0.2625, Val Loss: 1.3775
Epoch 2, Train Loss: 1.2006, Hits: 0.3676, F1: 0.3160, Val Loss: 1.2740
Epoch 3, Train Loss: 1.0188, Hits: 0.4513, F1: 0.3519, Val Loss: 1.2314
Epoch 4, Train Loss: 0.9293, Hits: 0.4936, F1: 0.3989, Val Loss: 1.1498
Epoch 5, Train Loss: 0.9133, Hits: 0.4779, F1: 0.3866, Val Loss: 1.1348
Epoch 6, Train Loss: 0.8633, Hits: 0.5487, F1: 0.4166, Val Loss: 1.1599
Epoch 7, Train Loss: 0.7915, Hits: 0.5625, F1: 0.4456, Val Loss: 1.0533
Epoch 8, Train Loss: 0.7394, Hits: 0.5561, F1: 0.4295, Val Loss: 1.0291
Epoch 9, Train Loss: 0.6951, Hits: 0.5524, F1: 0.4501, Val Loss: 1.0134
Epoch 10, Train Loss: 0.6860, Hits: 0.5653, F1: 0.4433, Val Loss: 1.0352
Epoch 11, Train Loss: 0.6526, Hits: 0.6452, F1: 0.4893, Val Loss: 1.0654
Epoch 12, Train Loss: 0.6569, Hits: 0.5919, F1: 0.4710, Val Loss: 1.0145
Epoch 13, Train Loss: 0.6110, Hits: 0.6360, F1: 0.5047, Val

In [134]:
item = training_data[-5]

In [135]:
item

{'idx': 1495,
 'initial_state': ({'B': 'A', 'D': 'B', 'A': 'sky', 'C': 'sky'},
  {'A': 'B', 'B': 'D', 'D': 'table', 'C': 'table'}),
 'goal_state': ({'D': 'A', 'A': 'C', 'B': 'D', 'C': 'sky'},
  {'A': 'D', 'C': 'A', 'D': 'B', 'B': 'table'}),
 'actions': [['unstack', 'A', 'B'],
  ['put down', 'A'],
  ['unstack', 'B', 'D'],
  ['put down', 'B'],
  ['pick up', 'D'],
  ['stack', 'D', 'B'],
  ['pick up', 'A'],
  ['stack', 'A', 'D'],
  ['pick up', 'C'],
  ['stack', 'C', 'A']],
 'group': [{'idx': 1495,
   'action': ['unstack', 'A', 'B'],
   'before_state': ({'B': 'A', 'D': 'B', 'A': 'sky', 'C': 'sky'},
    {'A': 'B', 'B': 'D', 'D': 'table', 'C': 'table'},
    None),
   'after_state': ({'B': 'sky', 'D': 'B', 'A': 'sky', 'C': 'sky'},
    {'A': 'table', 'B': 'D', 'D': 'table', 'C': 'table'},
    'A')},
  {'idx': 1495,
   'action': ['put down', 'A'],
   'before_state': ({'B': 'sky', 'D': 'B', 'A': 'sky', 'C': 'sky'},
    {'A': 'table', 'B': 'D', 'D': 'table', 'C': 'table'},
    'A'),
   'after_stat

In [136]:
def label_to_state(label):
    above = {}
    below = {}
    for i in range(n_blocks):
        below_block = int2block(label[i])
        above_block = int2block(label[i + n_blocks])
        block = int2block(i)

        above[block] = above_block
        below[block] = below_block

    return above, below, None

In [137]:

item_hidden_states = layer_hidden_states[0][item["idx"]]
pos = item["pos"]
inputs = torch.tensor(item_hidden_states[pos:pos + 5]).float().unsqueeze(0).to(device)

for j, probe in probes.items():
    with torch.no_grad():
        output = probe(inputs)
    # output = output.view(-1, n_blocks + 2, n_blocks * 2)
    preds = output.argmax(dim=-2).cpu().numpy().squeeze()

    print(label_to_state(preds))
    print(label_to_state(state_to_label(item["group"][j]["after_state"])))
    print()

({'A': 'sky', 'B': 'sky', 'C': 'B', 'D': 'B'}, {'A': 'table', 'B': 'D', 'C': 'table', 'D': 'table'}, None)
({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'B'}, {'A': 'table', 'B': 'D', 'C': 'table', 'D': 'table'}, None)

({'A': 'sky', 'B': 'sky', 'C': 'B', 'D': 'B'}, {'A': 'table', 'B': 'D', 'C': 'table', 'D': 'table'}, None)
({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'B'}, {'A': 'table', 'B': 'D', 'C': 'table', 'D': 'table'}, None)

({'A': 'sky', 'B': 'sky', 'C': 'B', 'D': 'sky'}, {'A': 'table', 'B': 'C', 'C': 'table', 'D': 'table'}, None)
({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'sky'}, {'A': 'table', 'B': 'table', 'C': 'table', 'D': 'table'}, None)

({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'sky'}, {'A': 'table', 'B': 'table', 'C': 'table', 'D': 'table'}, None)
({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'sky'}, {'A': 'table', 'B': 'table', 'C': 'table', 'D': 'table'}, None)

({'A': 'sky', 'B': 'sky', 'C': 'sky', 'D': 'sky'}, {'A': 'table', 'B': 'table', 'C': 'table', 'D': 'table'}, N

In [1]:
print(dataset[5]["generation"])

NameError: name 'dataset' is not defined

In [138]:
def find_index_before_block(text):
    text = text.lower()
    regex = r"\d+\.\s*(?:unstack|put down|pick up|stack)\s+(?:block\s+)?([a-z])"

    match = re.search(regex, text)
    
    return match.start(1) if match else None

for item in training_data:
    row = dataset[item["idx"]]
    tokens = tokenize_blocksworld_generation(tokenizer, row)[0]
    pos = item["pos"]

    # s = tokenizer.decode(tokens[pos:pos+20])

    # i = find_index_before_block(s)

    print(repr(tokenizer.decode(tokens[pos:pos+20])))
    print(repr(item["phrase"]), item["idx"])
    print(s[i:])
    print()


' C.\n3. Stack D on B.\n4. Stack A on D.\n\nBut initially, C'
'Let me try to outline the desired final stack: A on D on B on C. So the order from bottom to top is C, B, D, A.' 0


' A\n2. Put down D\n3. Unstack A from C\n4. Put down'
'So the plan would be:' 1


' C. Now, B is in hand, C is on D, D is on table, A'
'Wait, let me outline the steps:' 2


' D. Now, C is in hand, D is on B, and A is on table.'
'Wait, let me think step by step.' 3


' on top of Block A.\n\n2. Put down Block C.\n\n3. Unstack Block A from'
'So, the plan would be:' 4


' D. Now, C is in my hand, and D is on the table, clear.\n\n2'
'Let me think about how to do this step by step.' 5


' B: valid because D is on top of B and D is clear.\n\n2. Put down D'
'Let me go through each step:' 6


' on top of Block D. Now, Block B is in my hand, and Block D is now'
'Let me try that.' 7


' A.\n\n2. Stack B on C.\n\n3. Then stack D on B.\n\nBut to do'
'Alternatively, maybe I can rearrange the order:' 8


' B. (D in hand, B 

In [ ]:
]